In [ ]:
#librosa
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd

#keras
import os
import pandas as pd
import librosa
import glob 
import librosa.display
import random

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.callbacks import EarlyStopping

from keras import regularizers

from sklearn.preprocessing import LabelEncoder

import pandas as pd

from datetime import datetime

import os

import warnings
warnings.simplefilter('ignore')

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## EDA and dataset curation

In [ ]:
base_path = "/home/rishav/repo/ml/speaker-recognizer"
model_path = os.path.join(base_path, "models")

In [ ]:
!ls ../data/shivam_shukla/

**Visualize sound**

In [ ]:
sample_file = os.path.join(base_path, "data/shivam_shukla/TrainingAudio/Aadiksha-007/Aadiksha_1.wav")

In [ ]:
ipd.Audio(filename=sample_file)

**Plot sound**

In [ ]:
sample_audio, sr = librosa.load(sample_file)
plt.figure(figsize=(15, 17))

plt.subplot(3, 1, 1)
librosa.display.waveshow(sample_audio, alpha=0.5)
plt.ylim((-1, 1))
plt.title("Sample hindi audio")

In [ ]:
data_dir = "../data/shivam_shukla/"

In [ ]:
def get_file_path_and_speaker(data_dir):
    speaker_name_and_ids = {}
    speaker_counter = 0
    file_path_and_speaker = {}
    for root, dirs, files in os.walk(data_dir):
        path = root.split(os.sep)
        for file in files:
            speaker, file_path = file.split("_")[0].lower(), os.path.join(*path, file)            
            if speaker not in speaker_name_and_ids:
                speaker_counter += 1
                speaker_name_and_ids[speaker] = speaker_counter
            
            speaker_id = speaker_name_and_ids[speaker]
            file_path_and_speaker[file_path] = speaker_id
            
    
    speaker_df = pd.DataFrame.from_dict(file_path_and_speaker, orient='index')
    speaker_df['file_path'] = speaker_df.index
    speaker_df.reset_index(drop=True, inplace=True)
    speaker_df.rename(columns={0: 'speaker_id'}, inplace=True)
    
    return speaker_df, speaker_name_and_ids

    
speaker_df, speaker_name_and_ids = get_file_path_and_speaker(data_dir)

In [ ]:
print(f'''No. of speakers in the dataset : {len(speaker_df['speaker_id'].unique())}''')

In [ ]:
speaker_df[:2]

In [ ]:
len(speaker_name_and_ids)

## Features extraction

### MFCCs + resemblyzer embeddings + ecapa-tdnn embeddings


In [ ]:
# def extract_classical_features(file_path):
#     print(f'''File Path : {file_path}''')
#     # Loads the audio file as a floating point time series and assigns the default sample rate
#     # Sample rate is set to 22050 by default
#     X, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    
#     # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
#     mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    
#     # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
#     stft = np.abs(librosa.stft(X))
    
#     # Computes a chromagram from a waveform or power spectrogram.
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    
#     # Computes a mel-scaled spectrogram.
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    
#     # Computes spectral contrast
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    
#     # Computes the tonal centroid features (tonnetz)
#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
#     sr=sample_rate).T,axis=0)
#     return np.concatenate((mfccs, chroma, mel, contrast, tonnetz))

In [ ]:
# ran as a separate scipt

# encoder = VoiceEncoder()
# classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

# def extract_features(file_path):
#     print(f'''File Path : {file_path}''')
#     # Loads the audio file as a floating point time series and assigns the default sample rate
#     # Sample rate is set to 22050 by default
#     X, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    
#     # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
#     mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    
#     # resemblyzer embedding
#     wav = preprocess_wav(file_path)
#     resemblyzer_embedding = encoder.embed_utterance(wav)
    
#     # ecapa-tdnn embedding
#     ecapa_embeddings = classifier.encode_batch(signal)
#     ecapa_embeddings = np.array(embeddings)[0][0]
    
#     return np.concatenate((mfccs, resemblyzer_embedding, ecapa_embeddings))

In [ ]:
speaker_df = pd.read_pickle(os.path.join(base_path, "embeddings/embedding_resem_ecapa_mfcc_2022-11-11.pkl"))

In [ ]:
speaker_df = speaker_df.dropna()

In [ ]:
speaker_df.head()

In [ ]:
print(f''' Length of features : {len(speaker_df['features'][0])} ''')

In [ ]:
len(speaker_df), speaker_df[:2]

In [ ]:
len(speaker_df['speaker_id'].unique())

In [ ]:
single_speaker_df = speaker_df.groupby('speaker_id').agg({'file_path' : 'count'}).sort_values(by='file_path')
single_speaker_df.head()

In [ ]:
# speaker with only one files
speakers_with_1_file = single_speaker_df[single_speaker_df['file_path'] == 1].index.values.tolist()

In [ ]:
speaker_df  = speaker_df[~speaker_df['speaker_id'].isin(speakers_with_1_file)]
len(speaker_df['speaker_id'].unique())

In [ ]:
len(speaker_df)

In [ ]:
speaker_df.head()

In [ ]:
def split_list_col_to_multiple_cols(df, list_col_name):
    df = df.reset_index(drop=True)
    out_cols = [f'{list_col_name}_{col}' for col in range(len(df[list_col_name][0]))]

    split_df = pd.DataFrame(df[list_col_name].tolist(), columns=out_cols)
    df = pd.concat([df, split_df], axis=1)
    df = df.drop(columns=[list_col_name])
    return df

In [ ]:
speaker_df = split_list_col_to_multiple_cols(speaker_df, 'features')



The most efficient way:

1. Select last n columns

df1 = df.iloc[:,-n:]

2. Exclude last n columns

df1 = df.iloc[:,:-n]

In [ ]:
speaker_df.iloc[:,-40:]

In [ ]:
# mfcc_cols = [f"features_{counter}"  for counter in range(448, 488)]
# mfcc_cols = ["speaker_id", "file_path"] +  mfcc_cols
# speaker_df = speaker_df[mfcc_cols]

In [ ]:
# speaker_df = speaker_df.drop(columns=['file_path'])
speaker_df.head()

In [ ]:
speaker_df[speaker_df['speaker_id'].isna()]

In [ ]:
speaker_df_copy = speaker_df.copy()
speaker_df_copy = speaker_df_copy.drop(columns=['file_path'])
speaker_df_copy.shape

In [ ]:
mfcc_features = speaker_df_copy.iloc[:,-40:]
mfcc_features.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(speaker_df_copy,
                                                  speaker_df_copy['speaker_id'],
                                                  stratify=speaker_df_copy['speaker_id'],
                                                  test_size=0.20,
                                                  random_state=11)

In [ ]:
X_test_df = X_test.copy()

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
len(X_train['speaker_id'].unique()), len(X_test['speaker_id'].unique())

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
def get_cols_with_prefix(df, prefix):
    return [col for col in df.columns if col.startswith(prefix)]
    
x_cols = get_cols_with_prefix(X_train, 'features')    
X_train = X_train[x_cols]
X_test = X_test[x_cols]

**Normalize MFCC features**

In [ ]:
ss = StandardScaler()

In [ ]:
# mfcc cols
cols = ['features_'+ str(counter) for counter in range(448, 488)]

In [ ]:
X_train[cols].head()

In [ ]:
X_train = X_train[cols]
X_test = X_test[cols]

X_train[cols] = ss.fit_transform(X_train[cols])
X_test[cols] = ss.transform(X_test[cols])

In [ ]:
X_train.head()

In [ ]:
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_test = to_categorical(lb.fit_transform(y_test))

In [ ]:
X_train.min().min(), X_train.max().max(), X_test.min().min(), X_test.max().max()

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train = X_train.values
X_test = X_test.values

In [ ]:
print(pd.DataFrame(X_train).isna().sum().sum())
print(pd.DataFrame(X_test).isna().sum().sum())
print(pd.DataFrame(y_train).isna().sum().sum())
print(pd.DataFrame(y_test).isna().sum().sum())

In [ ]:
model = Sequential()
input_feature_size = 40
n_classes = 100

# initializer = tf.keras.initializers.GlorotNormal()
model.add(Dense(64, input_shape=(input_feature_size,), activation = 'relu'))
model.add(BatchNormalization())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.20))  

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(n_classes, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
print(model.summary())

In [ ]:
from keras.utils import plot_model

In [ ]:
dot_img_file = 'model_mfcc.png'
plot_model(model, to_file=dot_img_file, show_shapes=True, show_layer_activations=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=2000, 
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(235)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
predict_x = model.predict(X_test) 
classes_predicted = np.argmax(predict_x, axis=1)

In [ ]:
true_classes = np.argmax(y_test, axis=1)

In [ ]:
pred_df = pd.DataFrame({"true_label" : true_classes, "predicted_label" : classes_predicted})

In [ ]:
matched_df = pred_df[pred_df['true_label'] == pred_df['predicted_label']]
print(f'Accuracy : {len(matched_df)/ len(pred_df)}')

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
from itertools import cycle

In [ ]:
macro_roc_auc_ovo = roc_auc_score(y_test, predict_x, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    y_test, predict_x, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(y_test, predict_x, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    y_test, predict_x, multi_class="ovr", average="weighted"
)

print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)

In [ ]:
matched_df.head()

In [ ]:
matched_df['true_label'].values[i]

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(set(true_classes))):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], predict_x[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), predict_x.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
np.mean(list(roc_auc.values()))

In [ ]:
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

# 
# colors = cycle(["aqua", "darkorange", "cornflowerblue"])
# for i, color in zip(range(n_classes), colors):
#    plt.plot(
#        fpr[i],
#        tpr[i],
#        color=color,
#        lw=lw,
#        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
#    )
# 

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

In [ ]:
len(set(true_classes))